<a href="https://colab.research.google.com/github/sumithdcosta/Python/blob/master/BlockChain/python_blockchain_training_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%writefile hash_util.py
import hashlib as hl
import json

def hash_string_256(string):
  return hl.sha256(string).hexdigest()

def hash_block(block):
  #print(block)
  hashable_block = block.__dict__.copy()
  hashable_block['transactions'] = [tx.to_ordered_dict() for tx in hashable_block['transactions']]
  return hash_string_256(json.dumps(hashable_block, sort_keys=True).encode())


Writing hash_util.py


In [22]:
%%writefile block.py
from time import time

class Block:
  def __init__(self, index, previous_hash, transactions, proof, time=time()):
    self.index = index
    self.previous_hash = previous_hash
    self.transactions = transactions
    self.proof = proof
    self.time = time

  def __repr__(self):
    #return 'Index: {}, Previous hash: {}, Proof: {}, transactions: {}'.format(self.index, self.previous_hash, self.proof, self.transactions)
    return str(self.__dict__)

Overwriting block.py


In [23]:
%%writefile transaction.py
from collections import OrderedDict
class Transaction:
  def __init__(self, sender, recipient, amount):
    self.sender = sender
    self.recipient = recipient
    self.amount = amount

  def __repr__(self):
    return str(self.__dict__)

  def to_ordered_dict(self):
      return OrderedDict([('sender', self.sender),
                          ('recipient', self.recipient),
                          ('amount', self.amount)])

Overwriting transaction.py


In [24]:
%%writefile blockchain.py
# try except block : 
#####################################################
import functools
import hashlib as hl
import json
from collections import OrderedDict
import sys

from hash_util import hash_string_256, hash_block
from block import Block
from transaction import Transaction
from time import time

#global variable
MINING_REWARD = 10

blockchain = []

open_transactions = []


owner = 'Sumith'
participants = {owner}

def initialise():
  genesis_block = Block(index = 0,
                        previous_hash= '',
                        transactions = [],
                        proof = 100,
                        time = 0)

  blockchain = [genesis_block]
  return blockchain

def load_data():
  global blockchain
  global open_transactions 
  try:
    with open('blockchain.txt', mode='r') as f:
      file_content = f.readlines()

      blockchain = json.loads(file_content[0][:-1])
      updated_blockchain=[]
      for block in blockchain:
        converted_tx = [Transaction(tx['sender'],
                                    tx['recipient'],
                                    tx['amount']) for tx in block['transactions']]
        updated_block = Block(block['index'],
                              block['previous_hash'],
                              converted_tx,
                              block['proof'],
                              block['time'])
        updated_blockchain.append(updated_block)
      blockchain = updated_blockchain
      """blockchian = [(Block(block['index'],
                           block['previous_hash'],
                           (Transaction(tx['sender'],
                                        tx['recipient'],
                                        tx['amount']) for tx in block['transactions']),
                           block['proof'],
                           block['time'])
                      )for block in blockchain]"""
    
      open_transactions = json.loads(file_content[1])
      updated_transactions = []
      for tx in open_transactions:
        updated_transaction = Transaction(tx['sender'],
                                           tx['recipient'],
                                           tx['amount'])
        updated_transactions.append(updated_transaction) 
      open_transactions = updated_transactions
      """open_transactions = [Transaction(tx['sender'],
                                       tx['recipient'],
                                       tx['amount'])  for tx in open_transactions]"""
  except (IOError, IndexError):                    
    print('File not found!')
    blockchain = initialise()
  except ValueError:
    print('Value error!')
  except Exception as exc:
    print("Oops!",exc)
    print("Oops!",sys.exc_info()[0],"occured.")
    sys.exit(1)
  finally:
    print('Cleanup!')


load_data()

def save_data():
  try :
    with open('blockchain.txt', mode='w') as f:
      saveable_chain = [block.__dict__ for block in [Block(
          block_el.index,
          block_el.previous_hash,
          [tx.__dict__ for tx in block_el.transactions],
          block_el.proof,
          block_el.time
      ) for block_el in blockchain]]

      f.write(json.dumps(saveable_chain))
      f.write('\n')
      saveable_tx = [tx.__dict__ for tx in open_transactions]
      f.write(json.dumps(saveable_tx))
  except IOError:
    print('Error saving file!')

def get_last_blockchain_value():
  """ returns the last value of the current bloackchain """
  #function to return the value
  if len(blockchain) < 1 :
    return None
  return blockchain[-1]

def verify_transaction(transaction):
  sender_balance = get_balance(transaction.sender)
  return sender_balance >= transaction.amount


#adding kwargs to function and setting up default value
#transaction_amount is local varible
def add_transaction(recipient,sender=owner, amount=1.0):
  """ Appends a new value as well as the last blockchain value to teh block

  Arguments:
    : Sender :  sender of coins
    : recipent : recipient of coin
  """
  #appending values to blockchian and retaining the last value
  #transaction = {'sender': sender,
  #               'recipient': recipient, 
  #               'amount' : amount }
  transaction = Transaction(sender, recipient, amount)
  if verify_transaction(transaction):
    open_transactions.append(transaction)
    save_data()
    return True
  return False


def valid_proof(transactions, last_hash, proof):
  guess = (str([tx.to_ordered_dict() for tx in transactions]) + str(last_hash) + str(proof)).encode()
  guess_hash = hash_string_256(guess)
  #print(guess_hash)
  return guess_hash[0:2] == '00'

def proof_of_work():
  last_block = blockchain[-1]
  last_hash = hash_block(last_block)
  proof = 0
  while not valid_proof(open_transactions, last_hash, proof):
    proof += 1
  return proof
  

def get_balance(participant):
  tx_sender = [[tx.amount for tx in block.transactions if tx.sender == participant] for block in blockchain]
  open_tx_sender = [tx.amount for tx in open_transactions if tx.sender == participant]
  tx_sender.append(open_tx_sender)
  amount_sent = functools.reduce(lambda tx_sum, tx_amt: tx_sum + sum(tx_amt) if len(tx_amt) > 0 else tx_sum + 0, tx_sender,0)
  tx_recipient = [[tx.amount for tx in block.transactions if tx.recipient == participant] for block in blockchain]
  amount_received = functools.reduce(lambda tx_sum, tx_amt: tx_sum + sum(tx_amt) if len(tx_amt) > 0 else tx_sum + 0, tx_recipient,0)
  return  amount_received - amount_sent

def mine_block():
  last_block = blockchain[-1]
  #list comprehensions
  hashed_block = hash_block(last_block)
  proof = proof_of_work() 
  #reward_transaction = {
  #    'sender': 'MINING',
  #    'recipient': owner,
  #    'amount': MINING_REWARD
  #}
  reward_transaction = Transaction(sender = 'MINING', recipient = owner, amount = MINING_REWARD)
  copied_transactions = open_transactions[:]
  copied_transactions.append(reward_transaction)
  block = Block(index = len(blockchain),
                      previous_hash = hashed_block,
                      transactions = copied_transactions,
                      proof = proof,
                      time = 0)
  blockchain.append(block)
  #save_data()
  return True

#using function for reusable code.
def get_transaction_value():
  """ Retruns the input of the user(a new transaction amount) as a float
  """
  tx_recipient = input('Enter the recipent of the transaction: ') 
  tx_amount = float(input('Your trasaction amount please: ')) 
  return (tx_recipient, tx_amount)
  
def get_user_choice():
  user_input = input('Your Choice : ')
  return user_input

def print_blockchain_elements():
  #use for loop to output the blockchain list console
  for block in blockchain:
    print('outputting block : ', block)
  else:
    print('-' * 20)

def verify_chain():
  """verify current blockchain"""
  for (index, block) in enumerate(blockchain):
    if index == 0:
      continue
    if block.previous_hash != hash_block(blockchain[index -1]):
      return False
    if not valid_proof(block.transactions[:-1], block.previous_hash, block.proof):
      print('Prood of work invalid')
      return False
  return True

def verify_transactions():
  return all([verify_transaction(tx) for tx in open_transactions])
#  is_valid = True
#  for tx in open_transactions:
#    if verify_transaction(tx):
#      is_valid = True
#    else:
#      is_valid = False
  
waiting_for_input = True

while waiting_for_input:
  print('Please choose')
  print('1: Add a new transaction value')
  print('2: Mine a new block')
  print('3: Output the blockchain blocks')
  print('4: check transaction validity')
  print('q : Quit ')
  user_choice = get_user_choice()
  if user_choice == '1':
    tx_data = get_transaction_value()
    recipient, amount = tx_data
    if add_transaction(recipient, amount=amount):
      print('Added transaction')
    else:
      print('Tranaction failed')
    print(open_transactions)
  elif user_choice == '2':
    if mine_block():
      open_transactions = []
      save_data()
  elif user_choice == '3':
    print_blockchain_elements()
  elif user_choice == '4':
    if verify_transactions():
      print("All Transactions are valid")
    else:
      print('There are invalid transaction')
  elif user_choice == 'q':
    waiting_for_input = False
  else:
    print('Input was invalid, please pick a value from the list!')    
  
  if not verify_chain():
    print_blockchain_elements()
    print('Invalid block chain!')
    break
  print('Balance of {} : {:6.2f}'.format(owner, get_balance(owner)))  
  print('Choice registered !')
else:
  print('User left')

print('Done')


Overwriting blockchain.py


In [25]:
!python blockchain.py

Cleanup!
Please choose
1: Add a new transaction value
2: Mine a new block
3: Output the blockchain blocks
4: check transaction validity
q : Quit 
Your Choice : 3
outputting block :  {'index': 0, 'previous_hash': '', 'transactions': [], 'proof': 100, 'time': 0}
outputting block :  {'index': 1, 'previous_hash': 'f43bf96b7d7071552937a9fa0a82076d92554d2d593eaf76d8e77f19bd7cf5b8', 'transactions': [{'sender': 'MINING', 'recipient': 'Sumith', 'amount': 10}], 'proof': 195, 'time': 0}
--------------------
Balance of Sumith :  10.00
Choice registered !
Please choose
1: Add a new transaction value
2: Mine a new block
3: Output the blockchain blocks
4: check transaction validity
q : Quit 
Your Choice : q
Balance of Sumith :  10.00
Choice registered !
User left
Done


In [0]:
2

In [0]:
!rm blockchain.txt

In [0]:
!cat blockchain.txt

[{"index": 0, "previous_hash": "", "transactions": [], "proof": 100, "time": 0}, {"index": 1, "previous_hash": "f43bf96b7d7071552937a9fa0a82076d92554d2d593eaf76d8e77f19bd7cf5b8", "transactions": [{"sender": "MINING", "recipient": "Sumith", "amount": 10}], "proof": 195, "time": 0}, {"index": 2, "previous_hash": "a2e61e99c6c2d59fd55a172f4e8bda4eb57ed11eb95b15acca0aa2409b78a953", "transactions": [{"sender": "MINING", "recipient": "Sumith", "amount": 10}], "proof": 180, "time": 0}, {"index": 3, "previous_hash": "55c2ab59b701901fc72b4126af272830099eb79c5790ad7bb83db9ffb280b2c2", "transactions": [{"sender": "MINING", "recipient": "Sumith", "amount": 10}], "proof": 248, "time": 0}, {"index": 4, "previous_hash": "a771843b4253310565d6a9439171eb3bf5c6cb9b4438af30378b1cb4d4ac383d", "transactions": [{"sender": "MINING", "recipient": "Sumith", "amount": 10}], "proof": 136, "time": 0}, {"index": 5, "previous_hash": "39dd1672ace5ac86e4e1029ec8952b5b25de53e97dd9fcf7f0e3c29bee01a97d", "transactions": [

In [0]:
2